# This notebook is to plot geographical density of SCS counts over a given season:

In [17]:
### Importing necessary packages
import pandas as pd
from pathlib import Path
import re
import os
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import seaborn as sns

In [2]:
TIMEZONE_MAP = {
    # Eastern
    "EST": "US/Eastern", "EST-5": "US/Eastern", "EDT": "US/Eastern",
    "E": "US/Eastern", "ET": "US/Eastern",

    # Central
    "CST": "US/Central", "CST-6": "US/Central", "CDT": "US/Central",
    "C": "US/Central", "CT": "US/Central",

    # Mountain
    "MST": "US/Mountain", "MST-7": "US/Mountain", "MDT": "US/Mountain",
    "M": "US/Mountain", "MT": "US/Mountain",

    # Pacific
    "PST": "US/Pacific", "PST-8": "US/Pacific", "PDT": "US/Pacific",
    "P": "US/Pacific", "PT": "US/Pacific",

    # Alaska / Hawaii / Atlantic
    "AKST": "US/Alaska", "AST": "US/Alaska",  # AST can be Atlantic; check state if needed
    "HST": "US/Hawaii", "HAWAII": "US/Hawaii",
    "AST-4": "America/Puerto_Rico", "ATLANTIC": "America/Puerto_Rico"
}


def build_begin_datetime(df: pd.DataFrame) -> pd.Series:
    """
    Build a reliable datetime from BEGIN_YEARMONTH (YYYYMM),
    BEGIN_DAY, BEGIN_TIME (HHMM). Coerces bad values to NaT.
    """
    yyyymm = df["BEGIN_YEARMONTH"].astype("Int64").astype(str)
    day = df["BEGIN_DAY"].astype("Int64").astype(str).str.zfill(2)

    time = (
        df["BEGIN_TIME"]
        .astype("Int64")
        .fillna(0)
        .astype(str)
        .str.zfill(4)
    )

    dt_str = yyyymm + day + time
    return pd.to_datetime(dt_str, format="%Y%m%d%H%M", errors="coerce")


def standardize_and_convert_to_utc(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the dataframe with 'BEGIN_DT' (naive) and 'CZ_TIMEZONE',
    converts to UTC taking into account DST transitions.
    """
    df['clean_tz'] = df['CZ_TIMEZONE'].str.upper().str.strip().map(TIMEZONE_MAP)
    df['clean_tz'] = df['clean_tz'].fillna('UTC')

    utc_chunks = []

    for tz_name, group in df.groupby('clean_tz'):
        if tz_name == 'UTC':
            converted = group['BEGIN_DT'].dt.tz_localize('UTC')
        else:
            localized = group['BEGIN_DT'].dt.tz_localize(
                tz_name,
                ambiguous='NaT',
                nonexistent='shift_forward'
            )
            converted = localized.dt.tz_convert('UTC')

        group['BEGIN_DT_UTC'] = converted
        utc_chunks.append(group)

    df_utc = pd.concat(utc_chunks).sort_index()
    return df_utc.drop(columns=['clean_tz'])

In [3]:
# Loading in the data
data_dir = Path("/data1/lepique/stormevents_details/")  # adjust if needed
pattern = "StormEvents_details-ftp_v1.0_d*_c*.csv.gz"

storm_types = {"Tornado", "Hail", "Thunderstorm Wind"}
winter_months = {12, 1, 2, 3}

# Keeping only variables of immediate interest (can modify if needed)
keep_cols = [
    "EVENT_ID", "EPISODE_ID", "EVENT_TYPE",
    "BEGIN_DT",
    "STATE", "WFO",
    "BEGIN_LAT", "BEGIN_LON",
    "MAGNITUDE", "MAGNITUDE_TYPE",
]

files = sorted(data_dir.glob(pattern))
print(f"Found {len(files)} files")

dfs = []

for fp in files:
    m = re.search(r"_d(\d{4})_", fp.name)
    year = int(m.group(1)) if m else None

    print(f"Reading {fp.name} (year={year})")

    df = pd.read_csv(fp, compression="gzip", low_memory=False)

    # Ensure datetime is datetime
    df["BEGIN_DT"] = build_begin_datetime(df)
    df = df.dropna(subset=["BEGIN_DT"])

    # Drop rows with missing begin time (safer for dt.month)
    df = df.dropna(subset=["BEGIN_DATE_TIME"])

    # DJFM filter
    df = df[df["BEGIN_DT"].dt.month.isin(winter_months)]

    # December-year winter label
    dt = df["BEGIN_DT"]
    df["WINTER_SEASON"] = dt.dt.year - dt.dt.month.isin(winter_months).astype(int)

    # Storm-type filter
    df = df[df["EVENT_TYPE"].isin(storm_types)]

    # Optional: keep only selected columns
    # cols = ["WINTER_SEASON"] + keep_cols
    # df = df[cols].copy()

    # Optional: source file year
    df["SOURCE_FILE_YEAR"] = year

    dfs.append(df)

# Concatenate all years
df_winter_scs = pd.concat(dfs, ignore_index=True)

# Eliminate JFM 1950 because full winter season is not available
df_winter_scs = df_winter_scs[df_winter_scs["WINTER_SEASON"] >= 1950]

# Add UTC time column
df_winter_scs = standardize_and_convert_to_utc(df_winter_scs)

print("Done.")
print(df_winter_scs.shape)
print(df_winter_scs["EVENT_TYPE"].value_counts())
print(df_winter_scs["WINTER_SEASON"].min(), df_winter_scs["WINTER_SEASON"].max())

Found 76 files
Reading StormEvents_details-ftp_v1.0_d1950_c20250520.csv.gz (year=1950)
Reading StormEvents_details-ftp_v1.0_d1951_c20250520.csv.gz (year=1951)
Reading StormEvents_details-ftp_v1.0_d1952_c20250520.csv.gz (year=1952)
Reading StormEvents_details-ftp_v1.0_d1953_c20250520.csv.gz (year=1953)
Reading StormEvents_details-ftp_v1.0_d1954_c20250520.csv.gz (year=1954)
Reading StormEvents_details-ftp_v1.0_d1955_c20250520.csv.gz (year=1955)
Reading StormEvents_details-ftp_v1.0_d1956_c20250520.csv.gz (year=1956)
Reading StormEvents_details-ftp_v1.0_d1957_c20250520.csv.gz (year=1957)
Reading StormEvents_details-ftp_v1.0_d1958_c20250520.csv.gz (year=1958)
Reading StormEvents_details-ftp_v1.0_d1959_c20250520.csv.gz (year=1959)
Reading StormEvents_details-ftp_v1.0_d1960_c20250520.csv.gz (year=1960)
Reading StormEvents_details-ftp_v1.0_d1961_c20250520.csv.gz (year=1961)
Reading StormEvents_details-ftp_v1.0_d1962_c20250520.csv.gz (year=1962)
Reading StormEvents_details-ftp_v1.0_d1963_c20250

In [4]:
# Choose the DJFM winter season (December-year label)
# Example: 2010 means Dec 2010 – Mar 2011
season = 2010

season_subset = df_winter_scs[df_winter_scs["WINTER_SEASON"] == season]
print(f"Total DJFM reports in winter season {season}: {len(season_subset)}")
print(season_subset["EVENT_TYPE"].value_counts())

Total DJFM reports in winter season 2010: 3068
EVENT_TYPE
Thunderstorm Wind    1442
Hail                 1435
Tornado               191
Name: count, dtype: int64


In [15]:
def plot_season_density(df, season, event_type, bins=60, save_path=None):
    """
    Full-US cartopy map for one winter season and hazard type.
    Uses same projection/extent as plot_case, but shades density.
    """
    # Filter to this season + hazard
    et = df["EVENT_TYPE"].str.upper().fillna("OTHER")
    if event_type.lower().startswith("tornado"):
        mask = et.str.contains("TORNADO")
        cmap = "Reds"
    elif event_type.lower().startswith("hail"):
        mask = et.str.contains("HAIL")
        cmap = "Greens"
    else:  # Thunderstorm Wind
        mask = et.str.contains("THUNDERSTORM WIND") | et.str.contains("TSTM WIND")
        cmap = "Blues"

    season_df = df[(df["WINTER_SEASON"] == season) & mask].copy()
    season_df = season_df.dropna(subset=["BEGIN_LAT", "BEGIN_LON"])

    if season_df.empty:
        print(f"No {event_type} reports for WINTER_SEASON={season}")
        return

    # Same projection and CONUS extent as plot_case
    proj = ccrs.LambertConformal(
        central_longitude=-96, central_latitude=39,
        standard_parallels=(33, 45),
    )
    extent = [-130, -65, 23, 50]

    fig = plt.figure(figsize=(10, 6))
    ax = plt.axes(projection=proj)
    ax.set_extent(extent, crs=ccrs.PlateCarree())

    # Basemap features
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.6)
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=0.6)
    ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.4)
    ax.set_facecolor("white")

    # Fixed grid over CONUS to avoid weird artifacts
    ### This currently bins by degrees, but I would like to try binning by county, or area instead. 
    lon_min, lon_max, lat_min, lat_max = extent
    lon_bins = np.linspace(lon_min, lon_max, bins + 1)
    lat_bins = np.linspace(lat_min, lat_max, bins + 1)

    H, lon_edges, lat_edges = np.histogram2d(
        season_df["BEGIN_LON"].values,
        season_df["BEGIN_LAT"].values,
        bins=[lon_bins, lat_bins],
    )

    # Mask zero-count cells so background stays clean
    H_masked = np.ma.masked_where(H.T == 0, H.T)

    pcm = ax.pcolormesh(
        lon_edges,
        lat_edges,
        H_masked,
        transform=ccrs.PlateCarree(),
        cmap=cmap,
        shading="auto",
    )
    pcm.cmap.set_bad("none")  # masked cells transparent

    cb = plt.colorbar(pcm, ax=ax, orientation="vertical", pad=0.02, shrink=0.7)
    cb.set_label("Report count per grid cell")

    ax.set_title(f"{event_type} density\nDJFM winter season {season}", fontsize=13)
    plt.tight_layout()

    if save_path is not None:
        save_path = os.path.expanduser(save_path)
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved to: {os.path.abspath(save_path)}")
        plt.close(fig)
    else:
        plt.show()

In [ ]:
for hazard in ["Thunderstorm_Wind", "Hail", "Tornado"]:
    plot_season_density(df_winter_scs, season=1990, event_type=hazard, bins=60, save_path='~/circs_research/1990_'+hazard+'.png')

Saved to: /home1/lepique/circs_research/1990_Thunderstorm Wind.png
Saved to: /home1/lepique/circs_research/1990_Hail.png
Saved to: /home1/lepique/circs_research/1990_Tornado.png


In [19]:
plot_season_density(
df_winter_scs, 
season=2009, 
event_type='Hail', 
save_path='~/circs_research/hail_density_2009.png'
)


Saved to: /home1/lepique/circs_research/hail_density_2009.png


In [22]:
np.unique_counts((df_winter_scs['CZ_TYPE']))

UniqueCountsResult(values=array(['C', 'Z'], dtype=object), counts=array([109227,     42]))

In [23]:
## subsetting to only counties and not marine zones
df_winter_scs = df_winter_scs[df_winter_scs["CZ_TYPE"] == "C"]

In [24]:
##Converting state and county FIPS codes to strings so that I can combine them together to create unique COUNTY_FIPS codes
df_winter_scs =df_winter_scs[df_winter_scs["CZ_TYPE"] == "C"].copy()

df_winter_scs["STATE_FIPS"] = (
   df_winter_scs["STATE_FIPS"]
    .astype(float)
    .astype(int)
    .astype(str)
    .str.zfill(2)
)

df_winter_scs["CZ_FIPS"] = (
   df_winter_scs["CZ_FIPS"]
    .astype(float)
    .astype(int)
    .astype(str)
    .str.zfill(3)
)

df_winter_scs["COUNTY_FIPS"] = (
   df_winter_scs["STATE_FIPS"] +df_winter_scs["CZ_FIPS"]
)

In [25]:
#Verification:
print(df_winter_scs['COUNTY_FIPS'].iloc[0])
print(df_winter_scs["COUNTY_FIPS"].str.len().unique())

48225
[5]


In [31]:
print(type(df_winter_scs))
print(type(df_winter_scs['COUNTY_FIPS']))
print(type(df_winter_scs.groupby(["COUNTY_FIPS", "EVENT_TYPE"])))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [41]:
df_winter_scs.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,BEGIN_DT,WINTER_SEASON,SOURCE_FILE_YEAR,BEGIN_DT_UTC,COUNTY_FIPS
65,195103,28,510,195103,28,510,NaN,10120421,TEXAS,48,...,NaN,NaN,NaN,NaN,PUB,1951-03-28 05:10:00,1950,1951,1951-03-28 11:10:00+00:00,48225
66,195103,30,1500,195103,30,1500,NaN,10104933,PENNSYLVANIA,42,...,NaN,NaN,NaN,NaN,PUB,1951-03-30 15:00:00,1950,1951,1951-03-30 21:00:00+00:00,42001
67,195102,19,1830,195102,19,1830,NaN,10099493,OKLAHOMA,40,...,35.37,-98.13,NaN,NaN,PUB,1951-02-19 18:30:00,1950,1951,1951-02-20 00:30:00+00:00,40015
68,195102,19,1835,195102,19,1835,NaN,10099704,OKLAHOMA,40,...,35.83,-97.83,NaN,NaN,PUB,1951-02-19 18:35:00,1950,1951,1951-02-20 00:35:00+00:00,40017
69,195102,19,2200,195102,19,2200,NaN,10099705,OKLAHOMA,40,...,NaN,NaN,NaN,NaN,PUB,1951-02-19 22:00:00,1950,1951,1951-02-20 04:00:00+00:00,40027


### Now to try grouping and binning by County:

In [42]:
county_counts = (
    df_winter_scs.groupby(["COUNTY_FIPS","EVENT_TYPE", "WINTER_SEASON"])
    .size()
    .unstack(fill_value=0)  ##for making wide format
    .reset_index()
)

In [43]:
county_counts

WINTER_SEASON,COUNTY_FIPS,EVENT_TYPE,1950,1951,1952,1953,1954,1955,1956,1957,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,01000,Hail,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01000,Thunderstorm Wind,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01001,Hail,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,1
3,01001,Thunderstorm Wind,0,0,0,0,0,0,0,0,...,0,1,1,3,2,2,0,1,0,4
4,01001,Tornado,0,0,0,0,0,0,0,0,...,0,0,2,4,0,2,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7274,99127,Thunderstorm Wind,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7275,99127,Tornado,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7276,99133,Thunderstorm Wind,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7277,99139,Tornado,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
